In [167]:
import pandas as pd
import numpy as np
import pickle 
from datetime import datetime
from fuzzywuzzy import fuzz

with open('riaa_df.pkl', 'rb') as f:
        riaa_df = pickle.load(f)
with open('hot-100_chart_final.pkl', 'rb') as f:
        rec_df= pickle.load(f)     
with open('song_df_aggregate.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [ ]:
riaa_df["award_datetime"] = riaa_df["award_date"].apply(lambda x: datetime.strptime(x.strip(),"%B %d, %Y"))
song_df["spotify_album_release_date_datetime"] = song_df["spotify_album_release_date"].apply(lambda x: datetime.strptime(x.strip(),"%Y-%m-%d") if len(str(x))>8 else x)

In [ ]:
riaa_df["award"] = riaa_df["award"].apply(lambda x: x.strip())
keywords = ["Featuring","Feauring", "featuring", "&",","," X ", " x ","Presents", "From", "Or","And",]
for song in song_df.index:
    date = song_df.loc[song,"spotify_album_release_date_datetime"]
    artist_df = riaa_df[riaa_df["award_datetime"] <= date]
    artist = song_df.loc[song,"artist"]
    last_award_list = list()
    award_num_list = list()
    num_songs_awards = list()
    gold_count = list()
    platinum_count = list()
    
    artist_list = list()
    artist_list.append(artist)
    for keyword in keywords:
        artist_list.extend(artist.split(keyword))
    artist_list = list(set(artist_list))
    
    if len(str(date)) >8:
        for performer in artist_list:
            artist_df1 = artist_df[artist_df["artist"] == performer.strip()].sort_values("award_datetime",ascending=False)
            if len(artist_df1)>0:
                gold_count.append(artist_df1[artist_df1["award"] == "Gold"].count()["award"])
                artist_df1 = artist_df1.sort_values("award_datetime",ascending=False).drop_duplicates("songs")
                num_songs_awards.append(len(artist_df1["songs"].value_counts()))
                platinum_count.append(artist_df1[artist_df1["award"] == "Platinum"].sum()["award_num"])
                last_award_list.append(artist_df1.iloc[0]["award"])
                award_num_list.append(gold_count[-1] + platinum_count[-1])
            else:
                last_award_list.append(None)
                award_num_list.append(0)
                gold_count.append(0)
                platinum_count.append(0)
                num_songs_awards.append(0)
 
        max_index = award_num_list.index(max(award_num_list))
        song_df.loc[song,"last_award_type"] = last_award_list[max_index]
        song_df.loc[song,"award_num"] = award_num_list[max_index]
        song_df.loc[song,"gold_count"] = gold_count[max_index]
        song_df.loc[song,"platinum_count"] = platinum_count[max_index]
        song_df.loc[song,"num_songs_awards"] = num_songs_awards[max_index]
song_df["artist_has_award"] = song_df["award_num"] > 0

In [ ]:
song_df.groupby("album_label").count()["artist"].sort_values(ascending=False)

In [100]:
labels_occurence = list()
for i in np.asarray(song_df["album_label"]):
    if type(i) == str:
        label = i.split("/")
        for lab in label: 
            labels_occurence.append(lab.strip())

In [104]:
count_labels = list()
labels = set(labels_occurence)
for i in list(labels):
    count_labels.append([i,labels_occurence.count(i)])
label_df = pd.DataFrame(columns=["label","appearance_count"],data=count_labels)
label_df.sort_values("appearance_count",ascending=False)["appearance_count"][0:25].sum()

3936

In [105]:
label_df = label_df.sort_values("appearance_count",ascending=False)
labels = label_df["label"]
for label in label_df.index:
    label_stripped = label_df.loc[label,"label"]
    for i in labels:
        stopwords = ["Entertainment","Records","Recordings","LLC","Ltd."]
        i_stripped = i
        for word in stopwords:
            i_stripped = i_stripped.replace(word,"")
            label_stripped = label_stripped.replace(word,"")
        if  fuzz.partial_ratio(label_stripped,i_stripped) >= 90 and fuzz.ratio(label_stripped,i_stripped) >=65:
            if fuzz.ratio(label_df.loc[label,"label"],i) != 100:
                print(label_stripped,i_stripped,fuzz.ratio(label_df.loc[label,"label"],i))
            label_df.loc[label,"label_idx"] = i
            break

Atlantic  Atlantic 67
RCA  Label Nashville RCA  Label 77
Cash Money Cash Money  71
Universal Rec. Universal  84
Interscope  Interscope 71
Def Jam Def Jam  56
Universal Music Universal  69
Universal-Island   Universal  74
Republic Republic  67
Capitol  Nashville Capitol Nashville 81
Motown  Motown 60
Atlantic  UK Atlantic 59
Universal Republic  Universal  79
Sony Music  Sony Music Latin 70
Interscope * Interscope 69
Quality Control Music,  Quality Control Music 89
Nashville Arista Nashville 72
Sony Music UK Sony Music  59
Capitol Capitol  64
Grand Hustle,  Grand Hustle 83
Getting Out Our Dreams, Inc. Getting Out Our Dreams Inc. (G.O.O.D.) Music 75
Virgin EMI Virgin  58
Warner Bros. Warner  69
Geffen* Geffen 92
Rhino Atlantic Atlantic 73
EMI  Nashville Nashville 60
Young Money Young Money Ent. 81
Elektra Elektra  64
Syco Music UK Syco Music 87
Epic  Epic 50
2012 Ameritz Music  Ameritz Music Ltd 85
Cash Money  Inc. Cash Money  88
Capitol  (CAP) Capitol  83
Getting Out Our Dreams II Gettin

In [106]:
def label_mega_category(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row




        
label_df = label_df.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

In [107]:
label_df2 = label_df.groupby("label_idx").sum().sort_values("appearance_count",ascending=False)

def label_mega_category_groups(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row
label_df2 = label_df2.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

In [118]:
label_df = label_df.join(label_df2,on="label_idx",rsuffix="_idx")

In [119]:
label_df

,label,appearance_count,label_idx,label_category,appearance_count_idx,label_category_idx
651,Columbia,596,Columbia,Mega Label,599,Mega Label
595,Epic,351,Epic,Mega Label,366,Mega Label
973,Warner Records,268,Warner Records,Mega Label,288,Mega Label
58,RCA Records Label,223,RCA Records Label,Mega Label,317,Mega Label
550,Atlantic,211,Atlantic,Mega Label,467,Mega Label
...,...,...,...,...,...,...
261,4th & Broadway,1,4th & Broadway,Mini Label,1,Mini Label
262,OG Maco,1,OG Maco,Mini Label,1,Mini Label
267,Vertigo Berlin,1,Vertigo Berlin,Mini Label,1,Mini Label
200,101 Music,1,101 Music,Mini Label,1,Mini Label


In [131]:
for song in song_df.index:
    appearance_count = list()
    appearance_count_idx = list()
    label_category = list()
    label_category_idx = list()
    label = song_df.loc[song,"album_label"]
    if type(label) == str:
        label = label.split("/")
        for i in label:
            new_df = label_df[label_df["label"] == i.strip()]
            appearance_count_idx.append(new_df.iloc[0]["appearance_count_idx"])
            label_category_idx.append(new_df.iloc[0]["label_category_idx"])
            label_category.append(new_df.iloc[0]["label_category"])
            appearance_count.append(new_df.iloc[0]["appearance_count"])
        song_df.loc[song,"label_appearance_count"] = max(appearance_count)
        song_df.loc[song,"label_appearance_count_group"] = max(appearance_count_idx)
        song_df.loc[song,"label_category"] = label_category[appearance_count.index(max(appearance_count))]
        song_df.loc[song,"label_category_group"] = label_category_idx[appearance_count_idx.index(max(appearance_count_idx))]

In [135]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,firstrank,top10,top50,top25,top75,top5,label_appearance_count,label_category,label_appearance_count_group,label_category_group
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,85,True,True,True,True,True,1.0,Mini Label,1.0,Mini Label
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,7,True,True,True,True,True,107.0,Great Label,161.0,Great Label
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,2,True,True,True,True,True,76.0,Big Label,92.0,Big Label
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,50,True,True,True,True,True,211.0,Mega Label,467.0,Mega Label
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,15,True,True,True,True,True,70.0,Big Label,297.0,Mega Label
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8514,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,79,False,False,False,False,False,60.0,Big Label,92.0,Big Label
8515,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,93,False,False,False,False,False,69.0,Big Label,69.0,Big Label
8516,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,95,False,False,False,False,False,2.0,Mini Label,58.0,Big Label
8517,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,99,False,False,False,False,False,73.0,Big Label,73.0,Big Label


In [ ]:
rec_df["join"] = rec_df["artist"] + rec_df["title"]
song_df["join"] = song_df["artist"] + song_df["title"]
rec_df = rec_df.sort_values("date",ascending=False).drop_duplicates(["artist","title"])
rec_df.pop("artist")
rec_df.pop("isnew")
rec_df.pop("rank")
rec_df.pop("date")
rec_df.pop("title")
song_df = song_df.merge(rec_df,on="join")

In [ ]:
song_df["top10"] = song_df["bestrank"] < 10
song_df["top50"] = song_df["bestrank"] < 50
song_df["top25"] = song_df["bestrank"] < 25
song_df["top75"] = song_df["bestrank"] < 75
song_df["top5"] = song_df["bestrank"] < 5

In [173]:
with open('song_df_aggregate.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [162]:
rec_df2 = rec_df.sort_values("date").drop_duplicates(["artist","title"])
rec_df2["join"] = rec_df2["artist"] + rec_df2["title"]
#song_df["join"] = song_df["artist"] + song_df["title"]
rec_df2.pop("artist")
#rec_df2.pop("isnew")
rec_df2.pop("rank")
#rec_df2.pop("date")
rec_df2.pop("title")
rec_df2.pop("bestrank")
rec_df2.pop("firstrank")
#song_df = song_df.merge(rec_df2,on="join")



113199    100
113126     27
113127     28
113128     29
113129     30
         ... 
90         91
99        100
78         79
54         55
69         70
Name: firstrank, Length: 8519, dtype: int64

In [170]:
rec_df2["first_date"] = rec_df2["date"]
rec_df2.pop("date")

113199    1998-03-21
113126    1998-03-21
113127    1998-03-21
113128    1998-03-21
113129    1998-03-21
             ...    
90        2019-11-16
99        2019-11-16
78        2019-11-16
54        2019-11-16
69        2019-11-16
Name: date, Length: 8519, dtype: object

In [171]:
song_df = song_df.merge(rec_df2,on="join")

In [175]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,top50,top25,top75,top5,label_appearance_count,label_category,label_appearance_count_group,label_category_group,album_type,first_date
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,True,True,True,True,1.0,Mini Label,1.0,Mini Label,album,2019-05-25
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,True,True,True,True,107.0,Great Label,161.0,Great Label,album,2019-09-14
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,True,True,True,True,76.0,Big Label,92.0,Big Label,single,2019-07-06
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,True,True,True,True,211.0,Mega Label,467.0,Mega Label,album,2019-05-18
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,True,True,True,True,70.0,Big Label,297.0,Mega Label,single,2019-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8514,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,False,False,False,False,60.0,Big Label,92.0,Big Label,album,1998-03-21
8515,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,False,False,False,False,69.0,Big Label,69.0,Big Label,compilation,1998-03-21
8516,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,False,False,False,False,2.0,Mini Label,58.0,Big Label,album,1998-03-21
8517,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,False,False,False,False,73.0,Big Label,73.0,Big Label,album,1998-03-21
